# Getting point spreads from nflverse on GitHub

This notebook shows how we got the look-ahead point spread values that were used in our power ratings calculation.

By far the easiest way I know to get point spread data is to use the following file from nflverse on GitHub: [games.csv](https://github.com/nflverse/nfldata/blob/master/data/games.csv).

In [14]:
# for working with tabular data
import pandas as pd

We will read in this file directly from GitHub, but another option would be to download the file and read it in locally.

In [26]:
# read directly from GitHub
url = "https://raw.githubusercontent.com/nflverse/nfldata/master/data/games.csv"
df_full = pd.read_csv(url)

This is a large dataset with many more rows and columns than we will need.  (There are so many columns that we need to remove the limit imposed by pandas on the number of columns displayed.)  

In [27]:
pd.options.display.max_columns = None

Here are four random rows from the DataFrame.  This `sample` method is similar to the `head` method, but the `sample` method returns randomly selected rows rather than the first rows.

In [28]:
df_full.sample(4)

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
931,2002_11_CAR_TB,2002,REG,11,2002-11-17,Sunday,16:05,CAR,10.0,TB,23.0,Home,13.0,33.0,0.0,2002111705,18248.0,NaN,200211170tam,NaN,221117027.0,7,14,NaN,NaN,8.0,NaN,NaN,34.5,NaN,NaN,1,outdoors,grass,64.0,20.0,00-0012732,00-0008442,Rodney Peete,Brad Johnson,John Fox,Jon Gruden,Bob McElwee,TAM00,Raymond James Stadium
6172,2022_03_KC_IND,2022,REG,3,2022-09-25,Sunday,13:00,KC,17.0,IND,20.0,Home,3.0,37.0,0.0,2022092502,NaN,NaN,202209250clt,NaN,401437638.0,10,7,-199.0,177.0,-4.5,103.0,-114.0,51.0,-105.0,-107.0,0,open,fieldturf,NaN,NaN,00-0033873,00-0026143,Patrick Mahomes,Matt Ryan,Andy Reid,Frank Reich,Shawn Smith,IND00,Lucas Oil Stadium
1217,2003_12_CIN_SD,2003,REG,12,2003-11-23,Sunday,16:15,CIN,34.0,SD,27.0,Home,-7.0,61.0,0.0,2003112313,26449.0,NaN,200311230sdg,NaN,231123024.0,7,7,NaN,NaN,-3.0,NaN,NaN,43.0,NaN,NaN,0,outdoors,grass,67.0,9.0,00-0009311,00-0005363,Jon Kitna,Doug Flutie,Marvin Lewis,Marty Schottenheimer,Tony Corrente,SDG00,Qualcomm Stadium
1356,2004_03_DAL_WAS,2004,REG,3,2004-09-27,Monday,09:00,DAL,21.0,WAS,18.0,Home,-3.0,39.0,0.0,2004092700,26954.0,NaN,200409270was,NaN,240927028.0,8,8,NaN,NaN,1.5,NaN,NaN,35.5,NaN,NaN,1,outdoors,grass,72.0,7.0,00-0016193,00-0002110,Vinny Testaverde,Mark Brunell,Bill Parcells,Joe Gibbs,Gerry Austin,WAS00,FedExField


We will only need a few columns from this.  (In fact, I don't even think we will need the "week" column for these power ratings.)  We define a list `cols` containing the names of the columns we will use.

In [29]:
cols = ["home_team", "away_team", "location", "spread_line"]

We will also restrict to the season 2023.  The following says that we only want the rows for which the season is 2023, and we only want columns in the list `cols`.

In [32]:
df = df_full.loc[df["season"] == 2023, cols]

The resulting pandas DataFrame has 272 rows (one for each week of the regular season) and 4 columns.

In [33]:
df.shape

(272, 4)

Here we can see that the data contained in `df` is much more modest than what was contained in `df_full`.

In [34]:
df.sample(4)

,home_team,away_team,location,spread_line
6481,DAL,NE,Home,4.5
6653,TEN,SEA,Home,-2.0
6690,SF,LA,Home,7.0
6637,NO,NYG,Home,2.5


The "location" column might seem pointless, but it does contain some useful information.  The following is telling us that "Home" is the location for 267 of the rows, and "Neutral" is the location for the remaining 5 rows.

In [35]:
df["location"].value_counts()

location
Home       267
Neutral      5
Name: count, dtype: int64

Here are the neutral games in the dataset.  This approach to getting sub-DataFrames using the syntax `df[???]`, where `???` is something with Boolean (`True`/`False`) values, is very convenient and is called "Boolean indexing".

In [36]:
df[df["location"] == "Neutral"]

,home_team,away_team,location,spread_line
6470,JAX,ATL,Neutral,4.0
6486,BUF,JAX,Neutral,3.5
6500,TEN,BAL,Neutral,-5.5
6544,KC,MIA,Neutral,4.0
6558,NE,IND,Neutral,4.0


It will be more convenient for us to have this information represented differently.  Here we create a new column in `df` named "neutral" which contains `True` if the location is "Neutral" and contains `False` otherwise.

In [37]:
df["neutral"] = (df["location"] == "Neutral")

Here is an example of how the newly added column looks.  (The high numbers like `6468` on the left-hand side refer to the row numbers in the original full DataFrame.  We won't use them for anything.  If you instead wanted numbers from 0 to 271, you could use `df.reset_index(drop=True)`.  We won't bother, because when we save the resulting DataFrame, we will drop the index anyway.)

In [38]:
df.loc[6468:6472]

,home_team,away_team,location,spread_line,neutral
6468,CIN,LA,Home,7.5,False
6469,GB,DET,Home,-1.5,False
6470,JAX,ATL,Neutral,4.0,True
6471,BUF,MIA,Home,3.5,False
6472,CAR,MIN,Home,-1.5,False


Let's get rid of the old "location" column.  (The `axis=1` term tells pandas that we want to drop a column, rather than dropping a row, which would be `axis=0`.)

In [40]:
df_new = df.drop("location", axis=1)

In [41]:
df_new.loc[6468:6472]

,home_team,away_team,spread_line,neutral
6468,CIN,LA,7.5,False
6469,GB,DET,-1.5,False
6470,JAX,ATL,4.0,True
6471,BUF,MIA,3.5,False
6472,CAR,MIN,-1.5,False


Let's save this DataFrame as a csv file.  We'll include today's date in the file name, just in case we want access to multiple versions of this file later.

In [43]:
import time
today = time.strftime("%Y-%m-%d")

The following is an example of an f-string in Python, which enables the value of the variable `today` to be included as part of the written string.

In [46]:
filepath = f"data/spreads-{today}.csv"
filepath

'data/spreads-2023-08-10.csv'

Here we save the file.  Since we don't want those numbers like `6470` to be saved, we use `index=False` as one of our arguments.

In [47]:
df_new.to_csv(filename, index=False)